In [6]:
import pandas as pd
import numpy as np
import pandas_profiling 
from matplotlib import pyplot as plt
from datetime import datetime

from sklearn.model_selection import cross_val_score

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split



In [215]:
df = pd.read_csv(r"/Users/viveksavita/Downloads/Participants_Data_TGIH/train.csv")
df_test = pd.read_csv(r"/Users/viveksavita/Downloads/Participants_Data_TGIH/test.csv")

print(len(df_test))

print(df.head())
print(df_test.head())

122049
   InvoiceNo  StockCode  Description  Quantity          InvoiceDate  \
0       6141       1583          144         3  2011-05-06 16:54:00   
1       6349       1300         3682         6  2011-05-11 07:35:00   
2      16783       2178         1939         4  2011-11-20 13:20:00   
3      16971       2115         2983         1  2011-11-22 12:07:00   
4       6080       1210         2886        12  2011-05-06 09:00:00   

   UnitPrice  CustomerID  Country  
0       3.75     14056.0       35  
1       1.95     13098.0       35  
2       5.95     15044.0       35  
3       0.83     15525.0       35  
4       1.65     13952.0       35  
   InvoiceNo  StockCode  Description  Quantity          InvoiceDate  \
0       3139       1709         1079         3  2011-02-22 15:22:00   
1        617        510         3457         1  2010-12-08 12:46:00   
2      14653        604          694        36  2011-10-25 13:53:00   
3       8634       1478         3473         2  2011-06-27 12:38:0

In [216]:
def convert_date(df):
    df["InvoiceDate"] = df["InvoiceDate"].apply(lambda x : datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
    return df


df = convert_date(df)
df_test =     convert_date(df_test)      



In [217]:
df.drop_duplicates(inplace=True)
print(len(df))
df_1 = df.copy()
df_1.reset_index( inplace=True , drop=True)

df_1.drop(columns=['InvoiceNo'], axis=1, inplace=True)
df_1.describe()
print(df_1)

df_1_col = df_1.columns
print(df_1_col)

282159
        StockCode  Description  Quantity         InvoiceDate  UnitPrice  \
0            1583          144         3 2011-05-06 16:54:00       3.75   
1            1300         3682         6 2011-05-11 07:35:00       1.95   
2            2178         1939         4 2011-11-20 13:20:00       5.95   
3            2115         2983         1 2011-11-22 12:07:00       0.83   
4            1210         2886        12 2011-05-06 09:00:00       1.65   
...           ...          ...       ...                 ...        ...   
282154       1706         1081         6 2011-03-01 12:07:00       1.65   
282155       2184         2947        16 2011-10-13 15:08:00       0.65   
282156        667         3632         4 2011-05-16 12:59:00       8.50   
282157       1576         3089        48 2011-10-11 14:18:00       3.45   
282158       2603         1156         2 2011-08-22 13:30:00       0.75   

        CustomerID  Country  
0          14056.0       35  
1          13098.0       35  
2 

In [218]:
df_1["invoice_month"] = df_1["InvoiceDate"].dt.month
df_1["invoice_year"] = df_1["InvoiceDate"].dt.year

print(df_1)

        StockCode  Description  Quantity         InvoiceDate  UnitPrice  \
0            1583          144         3 2011-05-06 16:54:00       3.75   
1            1300         3682         6 2011-05-11 07:35:00       1.95   
2            2178         1939         4 2011-11-20 13:20:00       5.95   
3            2115         2983         1 2011-11-22 12:07:00       0.83   
4            1210         2886        12 2011-05-06 09:00:00       1.65   
...           ...          ...       ...                 ...        ...   
282154       1706         1081         6 2011-03-01 12:07:00       1.65   
282155       2184         2947        16 2011-10-13 15:08:00       0.65   
282156        667         3632         4 2011-05-16 12:59:00       8.50   
282157       1576         3089        48 2011-10-11 14:18:00       3.45   
282158       2603         1156         2 2011-08-22 13:30:00       0.75   

        CustomerID  Country  invoice_month  invoice_year  
0          14056.0       35             

In [219]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import KBinsDiscretizer

kbins = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
data_trans = kbins.fit_transform(df_label)
print(data_trans)
bins = pd.DataFrame(data_trans, columns=['label'])
bins.reset_index(inplace=True)

df_1 = pd.concat([df_1 , bins], axis=1 , join = 'inner' )
print(df_1)

[[7.]
 [5.]
 [8.]
 ...
 [9.]
 [7.]
 [1.]]
        StockCode  Description  Quantity         InvoiceDate  UnitPrice  \
0            1583          144         3 2011-05-06 16:54:00       3.75   
1            1300         3682         6 2011-05-11 07:35:00       1.95   
2            2178         1939         4 2011-11-20 13:20:00       5.95   
3            2115         2983         1 2011-11-22 12:07:00       0.83   
4            1210         2886        12 2011-05-06 09:00:00       1.65   
...           ...          ...       ...                 ...        ...   
282154       1706         1081         6 2011-03-01 12:07:00       1.65   
282155       2184         2947        16 2011-10-13 15:08:00       0.65   
282156        667         3632         4 2011-05-16 12:59:00       8.50   
282157       1576         3089        48 2011-10-11 14:18:00       3.45   
282158       2603         1156         2 2011-08-22 13:30:00       0.75   

        CustomerID  Country  invoice_month  invoice_year 

In [224]:
#print(df_1.groupby(['StockCode','label'])['label'].count())

df1 = pd.DataFrame(df_1.groupby(['StockCode','label','invoice_month'])['index'].count())
df1.reset_index(inplace=True)
print(df1)

print(df1.groupby(['StockCode','label'])['index'].sum())
df7 = pd.DataFrame(df_1.groupby(['StockCode','invoice_month'])['label'].max())
print(df7)


       StockCode  label  invoice_month  index
0              0    2.0              1     13
1              0    2.0              2      1
2              0    2.0              3      4
3              0    2.0              4      4
4              0    2.0             12     10
...          ...    ...            ...    ...
33988       3683    9.0              8     67
33989       3683    9.0              9     78
33990       3683    9.0             10     84
33991       3683    9.0             11    129
33992       3683    9.0             12     80

[33993 rows x 4 columns]
StockCode  label
0          2.0       32
1          0.0       13
           2.0        1
2          0.0       22
3          1.0        3
                   ... 
3683       5.0        1
           6.0        4
           7.0        9
           8.0       20
           9.0      801
Name: index, Length: 6523, dtype: int64
                         label
StockCode invoice_month       
0         1                2.0
        

In [235]:
# Finding the label for maximum count of the rows in stockcode
#print(type(df1.groupby(['StockCode','label'])['index'].sum()))

df2 = pd.DataFrame(df1.groupby(['StockCode','label'])['index'].sum())
df3 = pd.DataFrame(df2.groupby(['StockCode'])['index'].max())
df2.reset_index(inplace=True)
df3.reset_index(inplace=True)

df4 = (pd.merge(df3,df2,how='left' ,on=['StockCode','index']))
df4.drop(columns=['index'] , inplace=True,axis=1)
print(df2)
print(df3)
print(len(df4))

df4.drop_duplicates(inplace=True)
print(len(df4))

#df_cust_grp = df_1.groupby(['CustomerID'])['index'].sum()
#df4.to_csv(r"/Users/viveksavita/df4.csv")

      StockCode  label  index
0             0    2.0     32
1             1    0.0     13
2             1    2.0      1
3             2    0.0     22
4             3    1.0      3
...         ...    ...    ...
6518       3683    5.0      1
6519       3683    6.0      4
6520       3683    7.0      9
6521       3683    8.0     20
6522       3683    9.0    801

[6523 rows x 3 columns]
      StockCode  index
0             0     32
1             1     13
2             2     22
3             3      3
4             4      4
...         ...    ...
3614       3679     44
3615       3680     14
3616       3681    132
3617       3682      4
3618       3683    801

[3619 rows x 2 columns]
3659
3659


In [236]:
df_test1 = df_test.copy()

print(len(df_test1))


df_test1["invoice_month"] = df_test1["InvoiceDate"].dt.month
df_test1["invoice_year"] = df_test1["InvoiceDate"].dt.year

print(len(df_test1))
df_test1 = pd.merge(df7 ,df_test1, how ='right' , on =['StockCode','invoice_month'])
##df_test1.drop(columns=['index'], axis=1 , inplace=True)
df_test1.reset_index(inplace=True)
print(len(df_test1))
print(df_test1)
df_test1_col = df_test1.columns

122049
122049
122049
         index  StockCode  invoice_month  label  InvoiceNo  Description  \
0            0          0              1    2.0       2320         1698   
1            1          0              1    2.0       1852         1698   
2            2          0              1    2.0       1841         1698   
3            3          0              1    2.0       2336         1698   
4            4          0              1    2.0       1975         1698   
...        ...        ...            ...    ...        ...          ...   
122044  122044        480              7    NaN       9279         2080   
122045  122045       2747              1    NaN       1806         2809   
122046  122046       2858              9    NaN      12280         2565   
122047  122047       1865              9    NaN      12032         3252   
122048  122048       3377              8    NaN      10401         3222   

        Quantity         InvoiceDate  CustomerID  Country  invoice_year  
0   

In [237]:
 df6 = df4.drop_duplicates('StockCode').set_index('StockCode')['label']
print(df6)

StockCode
0       2.0
1       0.0
2       0.0
3       1.0
4       0.0
       ... 
3679    9.0
3680    9.0
3681    9.0
3682    0.0
3683    9.0
Name: label, Length: 3619, dtype: float64


In [238]:

df_test1['label']= df_test1.label.fillna(df_test1['StockCode'].map(df6))

df_test1['label']= df_test1.label.fillna(0)
print(df_test1)


         index  StockCode  invoice_month  label  InvoiceNo  Description  \
0            0          0              1    2.0       2320         1698   
1            1          0              1    2.0       1852         1698   
2            2          0              1    2.0       1841         1698   
3            3          0              1    2.0       2336         1698   
4            4          0              1    2.0       1975         1698   
...        ...        ...            ...    ...        ...          ...   
122044  122044        480              7    6.0       9279         2080   
122045  122045       2747              1    0.0       1806         2809   
122046  122046       2858              9    0.0      12280         2565   
122047  122047       1865              9    5.0      12032         3252   
122048  122048       3377              8    8.0      10401         3222   

        Quantity         InvoiceDate  CustomerID  Country  invoice_year  
0             14 2011-01-

In [239]:
df_test_temp = df_test1[df_test1.isna().any(axis=1)]


Empty DataFrame
Columns: [index, StockCode, invoice_month, label, InvoiceNo, Description, Quantity, InvoiceDate, CustomerID, Country, invoice_year]
Index: []


In [242]:
print(df_1.dtypes)
#print(df_test1)
#df_1.to_csv(r"/Users/viveksavita/train2.csv")

StockCode                 int64
Description               int64
Quantity                  int64
InvoiceDate      datetime64[ns]
UnitPrice               float64
CustomerID              float64
Country                   int64
invoice_month             int64
invoice_year              int64
index                     int64
label                   float64
dtype: object


In [268]:


cat_col = ['invoice_month','label','Country']#,'invoice_month','invoice_year']  #['StockCode','Description','CustomerID','Country' ] #,'invoice_month','invoice_year']
con_col = ['Quantity']
target = ['UnitPrice']


features = cat_col + con_col
print(features)

df_feature = df_1[features]
df_label = df_1[target]

print(df_feature.head())
print(df_label.head())

df_test_feature = df_test1[features]

print(df_test_feature)


['invoice_month', 'label', 'Country', 'Quantity']
   invoice_month  label  Country  Quantity
0              5    7.0       35         3
1              5    5.0       35         6
2             11    8.0       35         4
3             11    1.0       35         1
4              5    4.0       35        12
   UnitPrice
0       3.75
1       1.95
2       5.95
3       0.83
4       1.65
        invoice_month  label  Country  Quantity
0                   1    2.0       35        14
1                   1    2.0       35         6
2                   1    2.0       35         1
3                   1    2.0       13       120
4                   1    2.0       13        12
...               ...    ...      ...       ...
122044              7    6.0       35         6
122045              1    0.0       13        15
122046              9    0.0       35        17
122047              9    5.0       35        48
122048              8    8.0       35         1

[122049 rows x 4 columns]


In [270]:
 
df_features1 = pd.get_dummies(df_feature , columns=cat_col, drop_first=True)
s = StandardScaler()
#print(s.fit_transform(df_feature[['Quantity']]).reshape(-1,1))
df_features1['Quantity'] = s.fit_transform(df_feature[['Quantity']]).reshape(-1,1)
print(df_features1)


df_test_feature1 = pd.get_dummies(df_test_feature , columns=cat_col, drop_first=True)

df_test_feature1['Quantity'] = s.transform(df_test_feature[['Quantity']]).reshape(-1,1)
print(df_test_feature1)





gb = GradientBoostingRegressor()
gb.fit(df_features1,df_label)
y_pred= gb.predict(df_test_feature1)
final_pred = pd.DataFrame(y_pred, columns=['UnitPrice'])
print(len(final_pred))
print(final_pred)


        Quantity  invoice_month_2  invoice_month_3  invoice_month_4  \
0      -0.030729                0                0                0   
1      -0.020609                0                0                0   
2      -0.027355                0                0                0   
3      -0.037475                0                0                0   
4      -0.000371                0                0                0   
...          ...              ...              ...              ...   
282154 -0.020609                0                1                0   
282155  0.013122                0                0                0   
282156 -0.027355                0                0                0   
282157  0.121061                0                0                0   
282158 -0.034102                0                0                0   

        invoice_month_5  invoice_month_6  invoice_month_7  invoice_month_8  \
0                     1                0                0            

/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


122049
        UnitPrice
0        0.854441
1        1.031488
2        1.193524
3        0.713936
4        0.772843
...           ...
122044   2.155541
122045   1.009684
122046   1.091282
122047   1.556072
122048   4.874429

[122049 rows x 1 columns]


In [271]:
final_pred = pd.DataFrame(y_pred, columns=['UnitPrice'])
final_pred.to_csv(r"/Users/viveksavita/predictions_5.csv")

In [247]:
# Train and test split
X_train, X_test,y_train,y_test = train_test_split(df_feature,df_label, random_state=20)


from sklearn.preprocessing import StandardScaler , OneHotEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics 
from sklearn.model_selection import cross_val_predict


In [255]:
con_transformer = Pipeline(steps=[('scaler', StandardScaler())])
cat_transformer = Pipeline(steps =[('onehotencoding',OneHotEncoder(handle_unknown='ignore'))])


preprocessor = ColumnTransformer(
    transformers=[
    #('num', con_transformer , con_col),
    ('cat',cat_transformer,cat_col)
])



lr = Pipeline(
    steps=[('prep', preprocessor),
           ('regression',LinearRegression())    
])


regressors = [#LinearRegression(),
             #Lasso(),
             #ElasticNet(),
             #DecisionTreeRegressor()
             #KNeighborsRegressor(),
             GradientBoostingRegressor()
]

cv = KFold(n_splits=3, random_state=42, shuffle=False)

for reg in regressors:
    lr = Pipeline(
        steps=[('prep', preprocessor),
               ('regression',    reg)
              ])
    #lr.fit(X_train,y_train)
    #y_pred = lr.predict(X_test)
    y_pred = cross_val_predict(lr, df_feature, df_label, cv=5)
    
    #print(scores)
    print(reg,np.sqrt(metrics.mean_squared_error(df_label,y_pred)))
    #print(reg,np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
    


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:292: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False) 78.39850283591055


In [256]:

lr.fit(df_feature, df_label)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('onehotencoding',
                                                                   OneHotEncoder(categories='auto',
                                                                                 drop=None,
                                                                                 dtype=<class 'numpy.float64'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=True))],
                                                

In [257]:

final_pred = pd.DataFrame(lr.predict(df_impute_test), columns=['UnitPrice'])
print(final_pred)

ValueError: Number of features of the input must be equal to or greater than that of the fitted transformer. Transformer n_features is 4 and input n_features is 3.

In [31]:
lr = LinearRegression()
lr.fit(X_train,y_train)
#y_pred=lr.predict(X_test)

y_pred = cross_val_predict(lr, df_feature,df_label, cv=10)
print(np.sqrt(abs(cross_val_score(lr, df_feature,df_label, scoring = 'neg_mean_squared_error',cv=10 ))))
print(lr,np.sqrt(metrics.mean_squared_error(df_label,y_pred)))





[ 15.08739637  25.72352567  22.50367223  18.92129629 232.82326923
  16.04811621  11.21107857  31.26270039  50.07123503  25.66053819]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False) 77.77326294781457


In [35]:
lr.fit(df_feature,df_label)


from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
imputer.fit(df_feature)

df_impute_test = pd.DataFrame(imputer.transform(df_test_feature), columns = features)

print(df_impute_test)
final_pred = pd.DataFrame(lr.predict(df_impute_test), columns=['UnitPrice'])

df_test_final = pd.concat([df_test,final_pred], axis=1)

print(df_test_final)
df_test_final.to_csv(r"/Users/viveksavita/predictions_test.csv", index=False, header=True)

        Quantity  stockcodeencoded  countryencoded
0            3.0          3.745719        3.247452
1            1.0          1.973005        3.247452
2           36.0          5.895338        3.247452
3            2.0          0.824939        3.247452
4            1.0          1.635841        3.247452
...          ...               ...             ...
122044      12.0          0.431496        3.247452
122045      16.0          1.258647        4.799693
122046       6.0          1.261797        3.247452
122047       1.0          0.640821        3.247452
122048       3.0          9.862500        3.247452

[122049 rows x 3 columns]
        InvoiceNo  StockCode  Description  Quantity          InvoiceDate  \
0            3139       1709         1079         3  2011-02-22 15:22:00   
1             617        510         3457         1  2010-12-08 12:46:00   
2           14653        604          694        36  2011-10-25 13:53:00   
3            8634       1478         3473         2  2011